In [15]:
import pathlib
import xml.etree.ElementTree as ET

xml_file_path = '../000000.xml'
xml_root = ET.ElementTree(file=xml_file_path).getroot()
objects_node = xml_root.findall('object')
objects_node

[<Element 'object' at 0x000001461A1CDEA8>]